# Bayesian Optimazation Classification and Regression
> From now, stop using GridSearch and RandomSearch 

- toc: true 
- badges: true
- comments: true
- categories: [Bayesian]

In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from bayes_opt import BayesianOptimization
from sklearn.datasets import make_classification, make_regression
from sklearn.model_selection import cross_val_score

import warnings
warnings.simplefilter('ignore')

# Classification Problem Bayesian Optimazation

In [12]:
X, y = make_classification(n_samples=10000, n_features=10, n_classes=2)

We using the default hyperparameter to fit the data

In [13]:
rfc = RandomForestClassifier()
np.mean(cross_val_score(rfc, X, y, cv=5, scoring='roc_auc'))

0.9897763781114314

Default hyperparameter perform aroudn `0.98` ROC_AUC, next step we use `Bayesian Optimazation` to fine turn the hyperparameter. 

## Define the `blackBox` function

In [14]:
def rfc_cv(n_estimators, min_samples_split, max_features, max_depth):
    val = np.mean(cross_val_score(RandomForestClassifier(n_estimators=int(n_estimators), 
                                                 min_samples_split=int(min_samples_split),
                                                 max_features=min(max_features, 0.999),
                                                 max_depth=int(max_depth), random_state=42),
                         X, y, scoring='roc_auc', cv=5))
    
    return val

In [15]:
# define Bayesian Optimazation
rfc_bo = BayesianOptimization(
             rfc_cv,
             {'n_estimators': (10, 250),
             'min_samples_split': (2, 25),
             'max_features': (0.1, 0.999),
             'max_depth': (5, 30)})

In [16]:
# start the optimazation
rfc_bo.maximize()

|   iter    |  target   | max_depth | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------------------
|  1        |  0.9904   |  18.08    |  0.3725   |  2.766    |  249.1    |
|  2        |  0.9898   |  15.31    |  0.6883   |  24.87    |  84.16    |
|  3        |  0.99     |  12.87    |  0.416    |  24.62    |  51.27    |
|  4        |  0.9903   |  19.94    |  0.4735   |  5.652    |  244.5    |
|  5        |  0.9895   |  25.96    |  0.9587   |  12.6     |  244.6    |
|  6        |  0.9901   |  16.89    |  0.7025   |  2.421    |  244.6    |
|  7        |  0.9904   |  16.8     |  0.4017   |  6.605    |  248.0    |
|  8        |  0.9901   |  21.43    |  0.5085   |  6.751    |  249.4    |
|  9        |  0.9901   |  11.14    |  0.6435   |  4.991    |  248.4    |
|  10       |  0.9902   |  16.72    |  0.5588   |  9.387    |  242.5    |
|  11       |  0.9897   |  13.73    |  0.7405   |  12.09    |  248.8    |
|  12       |  0.9895   |  19.62    | 

In [17]:
# check the best hyperparameter
rfc_bo.max

{'target': 0.9905380799798376,
 'params': {'max_depth': 6.456055231994655,
  'max_features': 0.7869473158265811,
  'min_samples_split': 22.558253615710782,
  'n_estimators': 17.814015466174588}}

In [18]:
rfc_Optimazed = RandomForestClassifier(n_estimators=18, max_depth=6, max_features=0.78, min_samples_split=22)

In [19]:
np.mean(cross_val_score(rfc_Optimazed, X, y, cv=5, scoring='roc_auc'))

0.9900614797906387

* Original `roc_auc`: 0.989776
* Optimized `roc_auc`: 0.99006

# Regression Problem Bayesian Optimazation

In [3]:
X, y = make_regression(n_samples=10000, n_features=10)

In [4]:
rfe = RandomForestRegressor()
np.mean(cross_val_score(rfe, X, y, cv=5, scoring='neg_mean_squared_error'))

-1409.2889528620326

## Define the `blackbox` function

In [5]:
def rfe_cv(n_estimators, min_samples_split, max_features, max_depth):
    val = np.mean(cross_val_score(RandomForestRegressor(n_estimators=int(n_estimators), 
                                                 min_samples_split=int(min_samples_split),
                                                 max_features=min(max_features, 0.999),
                                                 max_depth=int(max_depth), random_state=42),
                         X, y, scoring='neg_mean_squared_error', cv=5))
    
    return val

In [ ]:
score = rfe_cv(n_estimators=100, min_samples_split=10, max_depth=6, max_features=0.78)
score

In [6]:
# define Bayesian Optimazation
rfe_bo = BayesianOptimization(
             rfe_cv,
             {'n_estimators': (10, 250),
             'min_samples_split': (2, 25),
             'max_features': (0.1, 0.999),
             'max_depth': (5, 30)})

In [8]:
# start the optimazation
rfe_bo.maximize()

|   iter    |  target   | max_depth | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------------------
|  1        | -2.702e+0 |  22.15    |  0.2902   |  20.4     |  211.9    |
|  2        | -2.788e+0 |  25.96    |  0.2216   |  22.83    |  166.3    |
|  3        | -1.651e+0 |  11.15    |  0.8612   |  10.86    |  153.0    |
|  4        | -5.608e+0 |  5.331    |  0.4747   |  10.79    |  49.97    |
|  5        | -1.862e+0 |  12.6     |  0.9883   |  21.26    |  124.8    |
|  6        | -5.684e+0 |  5.0      |  0.999    |  2.0      |  250.0    |
|  7        | -1.568e+0 |  12.54    |  0.9491   |  10.55    |  149.3    |
|  8        | -4.212e+0 |  30.0     |  0.1      |  2.0      |  117.6    |
|  9        | -5.682e+0 |  5.0      |  0.999    |  25.0     |  144.0    |
|  10       | -1.443e+0 |  20.34    |  0.758    |  8.014    |  154.0    |
|  11       | -1.412e+0 |  14.26    |  0.999    |  2.0      |  165.3    |
|  12       | -1.399e+0 |  30.0     | 

In [9]:
rfe_bo.max

{'target': -1390.7849548765093,
 'params': {'max_depth': 28.70255259053527,
  'max_features': 0.8468279746142502,
  'min_samples_split': 2.1219418980976834,
  'n_estimators': 140.748505191585}}

In [10]:
# use the best hyperparameter
rfe = RandomForestRegressor(n_estimators=140, max_depth=29, max_features=0.84, min_samples_split=2)

In [11]:
np.mean(cross_val_score(rfe, X, y, cv=5, scoring='neg_mean_squared_error'))

-1383.4479089516929

* Origin `neg_mean_squared_error`: -1409.2889528620326
* Optimazed `neg_mean_squared_error`: -1383.4479089516929